# Tribolium embryo morphometry
Authors: Robert Haase, Daniela Vorkel, June 2020

This is the pyclesperanto version of a workflow earlier [published for clij2](https://clij.github.io/clij2-docs/md/tribolium_morphometry/). 
[ImageJ Macro original](https://github.com/clij/clij2-docs/tree/master/src/main/macro/tribolium_morphometry.ijm)

This script is an example of heavy GPU-accelerated processing. It is recommended to use a dedicated
graphics card with at least 8 GB of GDDR6 memory. Otherwise, it may be quite slow.

Let's start by checking that pyclesperanto is installed and which GPU it uses.

In [103]:
import pyclesperanto_prototype as cle

print(cle.get_device().name)

gfx902


In [104]:
%gui qt

## Load a data set
The dataset is available [online](https://git.mpi-cbg.de/rhaase/clij2_example_data/blob/master/lund1051_resampled.tif).
It shows a *Tribolium castaneum* embryo, imaged by a custom light sheet microscope, at a wavelength of 488nm (Imaging credits: Daniela Vorkel, Myers lab, MPI CBG). 
The data set has been resampled to a voxel size of 1x1x1 microns. The embryo expresses nuclei-GFP. We will use the dataset to detect nuclei and to generate an estimated cell-segmentation.

All processing steps are performed in 3D space. For visualization purpose, we are using the maximum intensity projection in Z: 

In [105]:
from skimage.io import imread

image = imread('C:/structure/teaching/clij2_example_data/lund1051_resampled.tif')

# print out the spatial dimensions of the image
print(image.shape)

(213, 710, 355)


In [106]:
from skimage import data
import napari
viewer = napari.Viewer()

In [107]:
# voxel size z,y,x
calibration = [1, 1, 1]

In [108]:
import matplotlib.pyplot as plt

# convenience function for visualisation
def show(gpu_image):
    image = cle.pull_zyx(gpu_image)
    viewer.add_image(image, scale=calibration)
    
def show_labels(labels):
    viewer.add_labels(labels)

In [109]:
# push image to GPU memory and show it
gpu_input = cle.push_zyx(image)
# print(gpu_input)

show(gpu_input)

## Spot detection
After some noise removal/smoothing, we perform a local maximum detection:

In [110]:
# gaussian blur
sigma = 2.0
gpu_blurred = cle.gaussian_blur(gpu_input, sigma_x=sigma, sigma_y=sigma, sigma_z=sigma)

# detect maxima
gpu_detected_maxima = cle.detect_maxima_box(gpu_blurred)
show(gpu_detected_maxima)

## Spot curation
Now, we remove spots with values below a certain intensity and label the remaining spots.

In [111]:
# threshold
threshold = 300.0
gpu_thresholded = cle.greater_constant(gpu_blurred, constant=threshold)

# mask
gpu_masked_spots = cle.mask(gpu_detected_maxima, gpu_thresholded)

# label spots
gpu_labelled_spots = cle.connected_components_labeling_box(gpu_masked_spots)
show_labels(gpu_labelled_spots)

Let's see how many spots are left:

In [112]:
number_of_spots = int(cle.maximum_of_all_pixels(gpu_labelled_spots))
print("Number of detected spots: " + str(number_of_spots))

Number of detected spots: 1649


## Expanding labelled spots
Next, we spatially extend the labelled spots by applying a maximum filter.

In [113]:
# label map closing
number_of_dilations = 10


flip = cle.create_like(gpu_labelled_spots)
flop = cle.create_like(gpu_labelled_spots)
flag = cle.create([1,1,1])
cle.copy(gpu_labelled_spots, flip)


for i in range (0, number_of_dilations) :
    cle.onlyzero_overwrite_maximum_box(flip, flag, flop)
    cle.onlyzero_overwrite_maximum_diamond(flop, flag, flip)


Afterwards, we erode all labels in the map and get a final result of cell segementation.

In [114]:
flap = cle.greater_constant(flip, constant=1)
number_of_erosions = 4
for i in range(0, number_of_erosions):
    cle.erode_box(flap, flop)
    cle.erode_box(flop, flap)

gpu_labels = cle.mask(flip, flap)
show_labels(gpu_labels)

## Draw connectivity of the cells as a mesh
We then read out all current positions of detected nuclei as a pointlist to generate 
a distance matrix of all nuclei towards each other:

In [115]:
gpu_pointlist = cle.labelled_spots_to_pointlist(gpu_labelled_spots)
gpu_distance_matrix = cle.generate_distance_matrix(gpu_pointlist, gpu_pointlist)
# show(gpu_distance_matrix)

1649
3


c:\users\rober\miniconda3\lib\site-packages\pyopencl\__init__.py:253: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  "to see more.", CompilerWarning)


Starting from the label map of segmented cells, we generate a touch matrix:

In [116]:
gpu_touch_matrix = cle.generate_touch_matrix(gpu_labels)

# touch matrix:
# set the first column to zero to ignore all spots touching the background (background label 0, first column)
cle.set_column(gpu_touch_matrix, 0, 0)

# show(gpu_touch_matrix)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)

Using element by element multiplication of a distance matrix and a touch matrix, we calculate the length of 
each edge. We use this result to draw a mesh with a color gradient of distance (between 0 and 50 micron):

In [117]:
gpu_touch_matrix_with_distances = cle.multiply_images(gpu_touch_matrix, gpu_distance_matrix);

gpu_mesh = cle.create_like(gpu_input)
cle.set(gpu_mesh, 0)

cle.touch_matrix_to_mesh(gpu_pointlist, gpu_touch_matrix_with_distances, gpu_mesh);

show(gpu_mesh);